In [178]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import StandardScaler

In [179]:
from sklearn.linear_model import LinearRegression
import xgboost as xgb
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import RandomizedSearchCV

In [263]:
symbol_df = pd.read_csv("/Users/gaomo/Desktop/MLIP_data/symbols_valid_meta.csv")

In [264]:
vanguard_sp500 = pd.read_csv("/Users/gaomo/Desktop/MLIP_data/etfs/VOO.csv")
vanguard_sp500

,Date,Open,High,Low,Close,Adj Close,Volume
0,2010-09-09,102.500000,102.500000,101.139999,101.320000,83.143929,26500
1,2010-09-10,101.680000,101.860001,101.300003,101.779999,83.521400,8600
2,2010-09-13,102.959999,103.139999,102.500000,103.059998,84.571770,33700
3,2010-09-14,102.839996,103.480003,102.379997,103.040001,84.555374,59400
4,2010-09-15,102.620003,103.379997,102.400002,103.300003,84.768715,9200
...,...,...,...,...,...,...,...
2402,2020-03-26,229.270004,241.440002,228.850006,240.240005,240.240005,12411100
2403,2020-03-27,232.699997,239.639999,230.660004,232.860001,232.860001,11170600
2404,2020-03-30,234.919998,241.100006,232.970001,240.389999,240.389999,8727800
2405,2020-03-31,239.449997,241.940002,235.440002,236.820007,236.820007,10386300


## Trivial Non-ML

In [265]:
# open_mean = vanguard_sp500.Open.rolling(window=10).mean().dropna().to_numpy()
# high_mean = vanguard_sp500.High.rolling(window=10).mean().dropna().to_numpy()
# low_mean = vanguard_sp500.Low.rolling(window=10).mean().dropna().to_numpy()
close_meanpred = vanguard_sp500.Close.rolling(window=14).mean().dropna().to_numpy()

In [266]:
# open_val = vanguard_sp500.Open.to_numpy()[4:]
# high_val = vanguard_sp500.High.to_numpy()[4:]
# low_val = vanguard_sp500.Low.to_numpy()[4:]
close_val = vanguard_sp500.Close.to_numpy()[4:]

In [267]:
# print("open price mse is {}".format(mean_squared_error(open_val, open_mean)))
# print("high price mse is {}".format(mean_squared_error(high_val, high_mean)))
# print("low price mse is {}".format(mean_squared_error(low_val, low_mean)))
print("close price mae is {}".format(mean_absolute_error(close_val[-63:], close_meanpred[-63:])))

close price mae is 11.301984428007852


In [268]:
close_val[-64:-1]

array([295.79998779, 298.42001343, 296.23999023, 297.3500061 ,
       296.52999878, 298.05999756, 300.11999512, 299.26000977,
       301.23999023, 300.83999634, 301.51000977, 304.02999878,
       305.05999756, 304.42999268, 304.51000977, 304.80999756,
       302.02999878, 297.23001099, 300.45001221, 300.1499939 ,
       301.08999634, 295.69000244, 297.83999634, 302.42999268,
       305.8500061 , 306.89001465, 305.32998657, 307.51998901,
       308.07998657, 310.05999756, 309.76000977, 310.27999878,
       309.42999268, 310.92001343, 309.70001221, 306.48999023,
       296.23999023, 287.33999634, 286.07000732, 273.39001465,
       271.73999023, 283.86999512, 276.04000854, 287.48001099,
       277.8500061 , 273.19000244, 252.11000061, 263.80999756,
       250.72999573, 226.47999573, 247.63999939, 218.57000732,
       232.88000488, 220.1499939 , 220.33999634, 210.74000549,
       204.27000427, 223.75      , 226.61999512, 240.24000549,
       232.86000061, 240.38999939, 236.82000732])

In [269]:
vol = vanguard_sp500.Volume[-63:]
vol_lag = vanguard_sp500.Volume[-64:-1]
# 1 stands for increase in volume compared to the previous day, 0 otherwise
vol_label = [1 if i > j else 0 for i,j in zip(vol, vol_lag)]

In [270]:
clo = close_val[-64:-1]
clo_pred = close_meanpred[-63:]
# 1 stands for predicting increase in close prrice compared to the previous day, 0 otherwise
clo_label = [1 if i > j else 0 for i,j in zip(clo, clo_pred)]

In [271]:
correct_change_percentage = sum([1 if i == j else 0 for i,j in zip(vol_label, clo_label)])/len(clo_label)
correct_change_percentage


0.49206349206349204

## Simple ML

In [272]:
vanguard_sp500 = pd.read_csv("/Users/gaomo/Desktop/MLIP_data/etfs/VOO.csv")
vanguard_sp500['Date'] = pd.to_datetime(vanguard_sp500['Date'])

for i in range(1, 11):
    vanguard_sp500['lag_{}'.format(i)] = vanguard_sp500['Close'].shift(i)

vanguard_sp500.dropna(inplace=True)
vanguard_sp500

,Date,Open,High,Low,Close,Adj Close,Volume,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6,lag_7,lag_8,lag_9,lag_10
10,2010-09-23,103.320000,104.339996,103.120003,103.300003,84.768715,33300,104.160004,104.720001,105.040001,103.360001,103.260002,103.300003,103.040001,103.059998,101.779999,101.320000
11,2010-09-24,103.940002,104.900002,103.860001,104.839996,86.499718,24100,103.300003,104.160004,104.720001,105.040001,103.360001,103.260002,103.300003,103.040001,103.059998,101.779999
12,2010-09-27,104.980003,104.980003,104.339996,104.339996,86.087166,15000,104.839996,103.300003,104.160004,104.720001,105.040001,103.360001,103.260002,103.300003,103.040001,103.059998
13,2010-09-28,104.320000,105.019997,103.419998,105.019997,86.648224,49800,104.339996,104.839996,103.300003,104.160004,104.720001,105.040001,103.360001,103.260002,103.300003,103.040001
14,2010-09-29,104.580002,104.919998,104.220001,104.519997,86.235687,15900,105.019997,104.339996,104.839996,103.300003,104.160004,104.720001,105.040001,103.360001,103.260002,103.300003
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2402,2020-03-26,229.270004,241.440002,228.850006,240.240005,240.240005,12411100,226.619995,223.750000,204.270004,210.740005,220.339996,220.149994,232.880005,218.570007,247.639999,226.479996
2403,2020-03-27,232.699997,239.639999,230.660004,232.860001,232.860001,11170600,240.240005,226.619995,223.750000,204.270004,210.740005,220.339996,220.149994,232.880005,218.570007,247.639999
2404,2020-03-30,234.919998,241.100006,232.970001,240.389999,240.389999,8727800,232.860001,240.240005,226.619995,223.750000,204.270004,210.740005,220.339996,220.149994,232.880005,218.570007
2405,2020-03-31,239.449997,241.940002,235.440002,236.820007,236.820007,10386300,240.389999,232.860001,240.240005,226.619995,223.750000,204.270004,210.740005,220.339996,220.149994,232.880005


In [273]:
train = vanguard_sp500[vanguard_sp500.Date < '2020-01-01']
test = vanguard_sp500[vanguard_sp500.Date >= '2020-01-01']

In [274]:
X_train = train[['lag_{}'.format(i) for i in range(1, 11)]]
y_train = train['Close']
X_test = test[['lag_{}'.format(i) for i in range(1, 11)]]
y_test = test['Close']

In [240]:
X_test

,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6,lag_7,lag_8,lag_9,lag_10
2344,295.799988,295.040009,296.670013,296.670013,295.160004,295.160004,296.299988,294.799988,293.579987,293.559998
2345,298.420013,295.799988,295.040009,296.670013,296.670013,295.160004,295.160004,296.299988,294.799988,293.579987
2346,296.239990,298.420013,295.799988,295.040009,296.670013,296.670013,295.160004,295.160004,296.299988,294.799988
2347,297.350006,296.239990,298.420013,295.799988,295.040009,296.670013,296.670013,295.160004,295.160004,296.299988
2348,296.529999,297.350006,296.239990,298.420013,295.799988,295.040009,296.670013,296.670013,295.160004,295.160004
...,...,...,...,...,...,...,...,...,...,...
2402,226.619995,223.750000,204.270004,210.740005,220.339996,220.149994,232.880005,218.570007,247.639999,226.479996
2403,240.240005,226.619995,223.750000,204.270004,210.740005,220.339996,220.149994,232.880005,218.570007,247.639999
2404,232.860001,240.240005,226.619995,223.750000,204.270004,210.740005,220.339996,220.149994,232.880005,218.570007
2405,240.389999,232.860001,240.240005,226.619995,223.750000,204.270004,210.740005,220.339996,220.149994,232.880005


In [243]:
X_train

,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6,lag_7,lag_8,lag_9,lag_10
10,104.160004,104.720001,105.040001,103.360001,103.260002,103.300003,103.040001,103.059998,101.779999,101.320000
11,103.300003,104.160004,104.720001,105.040001,103.360001,103.260002,103.300003,103.040001,103.059998,101.779999
12,104.839996,103.300003,104.160004,104.720001,105.040001,103.360001,103.260002,103.300003,103.040001,103.059998
13,104.339996,104.839996,103.300003,104.160004,104.720001,105.040001,103.360001,103.260002,103.300003,103.040001
14,105.019997,104.339996,104.839996,103.300003,104.160004,104.720001,105.040001,103.360001,103.260002,103.300003
...,...,...,...,...,...,...,...,...,...,...
2339,295.160004,296.299988,294.799988,293.579987,293.559998,293.500000,291.440002,291.329987,288.790009,288.040009
2340,295.160004,295.160004,296.299988,294.799988,293.579987,293.559998,293.500000,291.440002,291.329987,288.790009
2341,296.670013,295.160004,295.160004,296.299988,294.799988,293.579987,293.559998,293.500000,291.440002,291.329987
2342,296.670013,296.670013,295.160004,295.160004,296.299988,294.799988,293.579987,293.559998,293.500000,291.440002


In [244]:
y_train

10      103.300003
11      104.839996
12      104.339996
13      105.019997
14      104.519997
           ...    
2339    295.160004
2340    296.670013
2341    296.670013
2342    295.040009
2343    295.799988
Name: Close, Length: 2334, dtype: float64

In [280]:
# For reflection questions
reg = xgb.XGBRegressor(n_estimators = 200,
                      max_depth = 4,
                      learning_rate = 0.14575757575757575).fit(X_train, y_train)
y_pred = reg.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
print("close price mae is {}".format(mae))


close price mae is 9.090111142113091


0.47619047619047616

In [281]:
reg = DecisionTreeRegressor().fit(X_train, y_train)
y_pred = reg.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
print("close price mae is {}".format(mae))

close price mae is 9.237466479104658


In [282]:
vol = vanguard_sp500.Volume[-63:]
vol_lag = vanguard_sp500.Volume[-64:-1]
# 1 stands for increase in volume compared to the previous day, 0 otherwise
vol_label = [1 if i > j else 0 for i,j in zip(vol, vol_lag)]

clo = close_val[-64:-1]
clo_pred = y_pred
# 1 stands for predicting increase in close prrice compared to the previous day, 0 otherwise
clo_label = [1 if i > j else 0 for i,j in zip(clo, clo_pred)]

correct_change_percentage = sum([1 if i == j else 0 for i,j in zip(vol_label, clo_label)])/len(clo_label)
correct_change_percentage

0.47619047619047616

## Data/Feature Manipulation

In [262]:
vanguard_sp500.dropna(inplace = True)

In [254]:
vanguard_sp500['slope'] = (vanguard_sp500.Open - vanguard_sp500.Close)/6.5
vanguard_sp500['std_volumn'] = (vanguard_sp500.Volume - vanguard_sp500.Volume.mean()) / vanguard_sp500.Volume.std()
vanguard_sp500

,Date,Open,High,Low,Close,Adj Close,Volume,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6,lag_7,lag_8,lag_9,lag_10,slope,std_volumn
10,2010-09-23,103.320000,104.339996,103.120003,103.300003,84.768715,33300,104.160004,104.720001,105.040001,103.360001,103.260002,103.300003,103.040001,103.059998,101.779999,101.320000,0.003076,-0.905680
11,2010-09-24,103.940002,104.900002,103.860001,104.839996,86.499718,24100,103.300003,104.160004,104.720001,105.040001,103.360001,103.260002,103.300003,103.040001,103.059998,101.779999,-0.138461,-0.910404
12,2010-09-27,104.980003,104.980003,104.339996,104.339996,86.087166,15000,104.839996,103.300003,104.160004,104.720001,105.040001,103.360001,103.260002,103.300003,103.040001,103.059998,0.098463,-0.915078
13,2010-09-28,104.320000,105.019997,103.419998,105.019997,86.648224,49800,104.339996,104.839996,103.300003,104.160004,104.720001,105.040001,103.360001,103.260002,103.300003,103.040001,-0.107692,-0.897206
14,2010-09-29,104.580002,104.919998,104.220001,104.519997,86.235687,15900,105.019997,104.339996,104.839996,103.300003,104.160004,104.720001,105.040001,103.360001,103.260002,103.300003,0.009232,-0.914615
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2402,2020-03-26,229.270004,241.440002,228.850006,240.240005,240.240005,12411100,226.619995,223.750000,204.270004,210.740005,220.339996,220.149994,232.880005,218.570007,247.639999,226.479996,-1.687692,5.450993
2403,2020-03-27,232.699997,239.639999,230.660004,232.860001,232.860001,11170600,240.240005,226.619995,223.750000,204.270004,210.740005,220.339996,220.149994,232.880005,218.570007,247.639999,-0.024616,4.813929
2404,2020-03-30,234.919998,241.100006,232.970001,240.389999,240.389999,8727800,232.860001,240.240005,226.619995,223.750000,204.270004,210.740005,220.339996,220.149994,232.880005,218.570007,-0.841539,3.559418
2405,2020-03-31,239.449997,241.940002,235.440002,236.820007,236.820007,10386300,240.389999,232.860001,240.240005,226.619995,223.750000,204.270004,210.740005,220.339996,220.149994,232.880005,0.404614,4.411148


In [255]:
train = vanguard_sp500[vanguard_sp500.Date < '2020-01-01']
test = vanguard_sp500[vanguard_sp500.Date >= '2020-01-01']
X_train = train[vanguard_sp500.columns[-12:]]
y_train = train['Close']
X_test = test[vanguard_sp500.columns[-12:]]
y_test = test['Close']

In [256]:
X_train

,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6,lag_7,lag_8,lag_9,lag_10,slope,std_volumn
10,104.160004,104.720001,105.040001,103.360001,103.260002,103.300003,103.040001,103.059998,101.779999,101.320000,0.003076,-0.905680
11,103.300003,104.160004,104.720001,105.040001,103.360001,103.260002,103.300003,103.040001,103.059998,101.779999,-0.138461,-0.910404
12,104.839996,103.300003,104.160004,104.720001,105.040001,103.360001,103.260002,103.300003,103.040001,103.059998,0.098463,-0.915078
13,104.339996,104.839996,103.300003,104.160004,104.720001,105.040001,103.360001,103.260002,103.300003,103.040001,-0.107692,-0.897206
14,105.019997,104.339996,104.839996,103.300003,104.160004,104.720001,105.040001,103.360001,103.260002,103.300003,0.009232,-0.914615
...,...,...,...,...,...,...,...,...,...,...,...,...
2339,295.160004,296.299988,294.799988,293.579987,293.559998,293.500000,291.440002,291.329987,288.790009,288.040009,0.030766,-0.252130
2340,295.160004,295.160004,296.299988,294.799988,293.579987,293.559998,293.500000,291.440002,291.329987,288.790009,-0.170772,0.304768
2341,296.670013,295.160004,295.160004,296.299988,294.799988,293.579987,293.559998,293.500000,291.440002,291.329987,0.129230,0.220083
2342,296.670013,296.670013,295.160004,295.160004,296.299988,294.799988,293.579987,293.559998,293.500000,291.440002,0.263076,0.281709


In [257]:
reg = DecisionTreeRegressor().fit(X_train, y_train)
y_pred = reg.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
print("close price mae is {}".format(mae))

close price mae is 8.298733423626608


In [258]:
vol = vanguard_sp500.Volume[-63:]
vol_lag = vanguard_sp500.Volume[-64:-1]
# 1 stands for increase in volume compared to the previous day, 0 otherwise
vol_label = [1 if i > j else 0 for i,j in zip(vol, vol_lag)]

clo = close_val[-64:-1]
clo_pred = y_pred
# 1 stands for predicting increase in close prrice compared to the previous day, 0 otherwise
clo_label = [1 if i > j else 0 for i,j in zip(clo, clo_pred)]

correct_change_percentage = sum([1 if i == j else 0 for i,j in zip(vol_label, clo_label)])/len(clo_label)
correct_change_percentage

0.5873015873015873

## Model Selection

In [114]:
xg = xgb.XGBRegressor()

params = {
    "max_depth": np.arange(2, 10),  
    "learning_rate": np.linspace(0.01, 0.15, 100), 
    "n_estimators": np.arange(100, 1000, 50) 
}

clf = RandomizedSearchCV(xg, params, cv=3)
search = clf.fit(X_train, y_train)
print(search.best_params_)


{'n_estimators': 200, 'max_depth': 4, 'learning_rate': 0.14575757575757575}


In [259]:
reg = xgb.XGBRegressor(n_estimators = 200,
                      max_depth = 4,
                      learning_rate = 0.14575757575757575).fit(X_train, y_train)
y_pred = reg.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
print("close price mae is {}".format(mae))

close price mae is 7.8407113211495485


In [260]:
vol = vanguard_sp500.Volume[-63:]
vol_lag = vanguard_sp500.Volume[-64:-1]
# 1 stands for increase in volume compared to the previous day, 0 otherwise
vol_label = [1 if i > j else 0 for i,j in zip(vol, vol_lag)]

clo = close_val[-64:-1]
clo_pred = y_pred
# 1 stands for predicting increase in close prrice compared to the previous day, 0 otherwise
clo_label = [1 if i > j else 0 for i,j in zip(clo, clo_pred)]

correct_change_percentage = sum([1 if i == j else 0 for i,j in zip(vol_label, clo_label)])/len(clo_label)
correct_change_percentage

0.5396825396825397